In [1]:
import datetime
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

### Introduction

This project is about analyzing crimes and house prices in Chicago. Based on the different house prices of different neighborhoods and the crime occurrence in those areas, our goal is to determine whether there is correlation between the crime rates and house prices in different neighborhoods. We have analyzed chicago crime data and median sale price of houses of different neighborhoods in Chicago. We have calculated the correlations between change in house prices and number of crimes in different neighborhoods from 2012-2018 and we came to a conclusion on whether or not there is a connection between them based on available data. Furthurmore, we have predicted house prices and number of crimes for two more years and shown that the predicted data also supports our conclusion. 

### Data Cleaning

We have used two different datasets for this project:

1. Chicago crime data from city of chicago website: https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2/data
2. Chicago house prices data from redfin: https://www.redfin.com/blog/data-center

After getting the house price data, we removed those column where there was no value available. We also planned to work with price per square feet value. So we keep Price per square and sale price and filtered out the rest.

Since we have house price of chicago neighborhood from 2012-2018, we get rid of all the crimes which occurred before 2012 and after 2019. We also got rid of the rows with null values. 

### Chicago Crime Data:

